In [1]:
# pip install sweetviz
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import sweetviz as sw
import seaborn as sns
sns.set()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 78.8 MB/s eta 0:00:00


In [2]:
schiller_url='https://github.com/schutera/tiny_schiller/blob/main/tiny_schiller.txt'
filepath=keras.utils.get_file('tiny_schiller.txt',schiller_url)
with open(filepath) as f:
    schiller_text=f.read()

   8192/Unknown - 0s 0us/step

In [4]:
tokenizer=keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(schiller_text)

max_id=len(tokenizer.word_index)
dataset_size=tokenizer.document_count
[encoded]=np.array(tokenizer.texts_to_sequences([schiller_text]))-1

In [5]:
train_size=dataset_size*90//100
dataset=tf.data.Dataset.from_tensor_slices(encoded[:train_size])

n_steps=100
window_length=n_steps+1
dataset=dataset.repeat().window(window_length,shift=1,drop_remainder=True)

dataset=dataset.flat_map(lambda window: window.batch(window_length))

batch_size=32
dataset=dataset.shuffle(10000).batch(batch_size)
dataset=dataset.map(lambda windows: (windows[:,:-1],windows[:,1:]))
dataset=dataset.map(lambda X_batch,Y_batch: (tf.one_hot(X_batch,depth=max_id),Y_batch))
dataset=dataset.prefetch(1)

In [6]:
model=keras.models.Sequential()
model.add(keras.layers.GRU(128,return_sequences=True,input_shape=[None,max_id]))
model.add(keras.layers.GRU(128,return_sequences=True))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation='softmax')))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')
history=model.fit(dataset,steps_per_epoch=train_size // batch_size,epochs=1)

In [8]:
def preprocess(texts):
    X=np.array(tokenizer.texts_to_sequences(texts))-1
    return tf.one_hot(X,max_id)

def next_char(text,temperature=1):
    X_new=preprocess([text])
    y_proba=model.predict(X_new)[0,-1:,:]
    rescaled_logits=tf.math.log(y_proba)/temperature
    char_id=tf.random.categorical(rescaled_logits,num_samples=1)+1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text,n_chars=50,temperature=1):
    for _ in range(n_chars):
        text+=next_char(text,temperature)
    return text

In [9]:
print("Some predicted texts for word 'Mark' are as follows:\n ")
for i in range(3):
  print(complete_text('Mark'))
  print()

Some predicted texts for word 'Mark' are as follows:
 
1/1 [==============================] - 0s 45ms/step
Markank ginhihotacanave4andevers ht/puen wmode-hesmes"

1/1 [==============================] - 0s 30ms/step
Markscrore;· content="tnmos_rifhub_asterianax_n0eco_re

1/1 [==============================] - 0s 27ms/step
Mark-coms" / ml serel="http ://plicrelrihlbgiadiass_pt

